ANNOTATION CHANGER

In [6]:
import json

def coco_to_vgg(coco_annotation):
    vgg_annotation = {}

    for image_info in coco_annotation['images']:
        image_id = str(image_info['id'])
        vgg_annotation[image_id] = {
            "filename": image_info['file_name'],
            "size": image_info['height'] * image_info['width'],
            "regions": [],
            "file_attributes": {
                "caption": "",
                "public_domain": "no",
                "image_url": ""
            }
        }

        for annotation in coco_annotation['annotations']:
            if annotation['image_id'] == int(image_id):
                region = {
                    "shape_attributes": {
                        "name": "polygon",
                        "all_points_x": annotation['segmentation'][0][0::2],
                        "all_points_y": annotation['segmentation'][0][1::2],
                    },
                    "region_attributes": {
                        "image_quality": {
                            "good": True,
                            "frontal": True,
                            "good_illumination": True
                        },
                        "names": coco_annotation['categories'][annotation['category_id']]['name']
                    }
                }
                vgg_annotation[image_id]['regions'].append(region)

    return vgg_annotation

def convert_and_save(input_path, output_path):
    with open(input_path, 'r') as file:
        coco_annotation = json.load(file)

    vgg_annotation = coco_to_vgg(coco_annotation)

    with open(output_path, 'w') as file:
        json.dump(vgg_annotation, file, indent=2)

# Input and output file paths
input_file_path = 'C:\\Users\\afiqe\\Downloads\\FYP DATASET LAMA.v10i.coco-segmentation\\dataset\\test\\_annotations.coco.json'
output_file_path = 'C:\\Users\\afiqe\\Downloads\\FYP DATASET LAMA.v10i.coco-segmentation\\dataset\\test\\_annotations1.coco.json'

# Convert COCO to VGG and save to the output file
convert_and_save(input_file_path, output_file_path)


In [8]:
import json

def count_images(annotation_data):
    return len(annotation_data)

def main():
    # Replace 'path_to_your_annotation_file.json' with the actual path to your annotation file
    annotation_file_path = 'C:\\Users\\afiqe\\Downloads\\FYP DATASET LAMA.v10i.coco-segmentation\\dataset\\test\\_annotations1.coco.json'

    try:
        with open(annotation_file_path, 'r') as f:
            annotation_data = json.load(f)

            # Call the function to count images
            num_images = count_images(annotation_data)

            print(f"Number of images: {num_images}")

    except FileNotFoundError:
        print(f"Error: File '{annotation_file_path}' not found.")
    except json.JSONDecodeError:
        print(f"Error: Unable to decode JSON from '{annotation_file_path}'. Please check if the file is valid JSON.")

if __name__ == "__main__":
    main()


Number of images: 38


In [ ]:
import json
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

def load_coco_annotation(file_path):
    with open(file_path, 'r') as file:
        coco_annotation = json.load(file)
    return coco_annotation

def visualize_masks(image, masks, class_ids, class_names):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(image)

    unique_class_ids = list(set(class_ids))
    for i, class_id in enumerate(unique_class_ids):
        mask = masks[:, :, class_ids == class_id]
        color = plt.cm.get_cmap('tab20')(i / len(unique_class_ids))

        for j in range(mask.shape[2]):
            contour, _ = cv2.findContours(mask[:, :, j].astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for c in contour:
                polygon = Polygon(c.reshape((-1, 2)), fill=False, edgecolor=color, linewidth=2)
                ax.add_patch(polygon)

    ax.set_title('Mask Visualization')
    plt.axis('off')
    plt.show()

# Example usage
coco_annotation_path = 'path/to/your/annotation.json'
coco_annotation = load_coco_annotation(coco_annotation_path)

# Replace image_id with the desired image index you want to visualize
image_id = 0
image_info = coco_annotation['images'][image_id]
image = cv2.imread(image_info['file_name'])
masks, class_ids = your_dataset.load_mask(image_id)  # Replace with your actual load_mask function

# Replace your_dataset.class_names with the actual class names
visualize_masks(image, masks, class_ids, your_dataset.class_names)


RESIZE IMAGE

In [8]:
from PIL import Image
import os

input_folder = 'C:\\Users\\afiqe\\Downloads\\Dataset10Class'
output_folder = 'C:\\Users\\afiqe\\Downloads\\DatasetPreProc'
target_size = (200, 200)  # Set your desired size

for class_name in os.listdir(input_folder):
    class_input_folder = os.path.join(input_folder, class_name)
    class_output_folder = os.path.join(output_folder, class_name)

    if os.path.isdir(class_input_folder):
        os.makedirs(class_output_folder, exist_ok=True)

        for filename in os.listdir(class_input_folder):
            if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
                image_path = os.path.join(class_input_folder, filename)
                image = Image.open(image_path)

                # Convert to 'RGB' mode if the image has a palette-based mode ('P')
                if image.mode == 'P':
                    image = image.convert('RGB')
                    
                if image.mode == 'RGBA':
                    image = image.convert('RGB')

                # Resize the image
                image = image.resize(target_size, Image.ANTIALIAS)

                # Save the resized image
                output_path = os.path.join(class_output_folder, filename)
                image.save(output_path)


C:\Users\afiqe\AppData\Local\Temp\ipykernel_14580\2306709436.py:28: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize(target_size, Image.ANTIALIAS)
E:\anaconda3\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


SPLIT

In [1]:
import os
import shutil
import random

# Define the source directory containing your dataset
source_dir = 'C:\\Users\\afiqe\\Downloads\\FYP\\Dataset\\Dataset10Class'

# Define the target directories for training, validation, and testing
train_dir = 'C:\\Users\\afiqe\\Downloads\\FYP\\Dataset\\Dataset60\\train'
val_dir = 'C:\\Users\\afiqe\\Downloads\\FYP\\Dataset\\Dataset60\\val'
test_dir = 'C:\\Users\\afiqe\\Downloads\\FYP\\Dataset\\Dataset60\\test'

# Create target directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Set the ratio for splitting the data
train_ratio = 0.6
val_ratio = 0.35
test_ratio = 0.05

# List the classes (subdirectories) in the source directory
class_names = os.listdir(source_dir)

# Loop through each class and split the data
for class_name in class_names:
    class_dir = os.path.join(source_dir, class_name)
    file_list = os.listdir(class_dir)
    
    random.shuffle(file_list)  # Shuffle the files

    num_files = len(file_list)
    num_train = int(train_ratio * num_files)
    num_val = int(val_ratio * num_files)
    
    train_files = file_list[:num_train]
    val_files = file_list[num_train:num_train + num_val]
    test_files = file_list[num_train + num_val:]

    # Copy files to the target directories
    for file in train_files:
        src = os.path.join(class_dir, file)
        dest = os.path.join(train_dir, class_name, file)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        shutil.copy(src, dest)

    for file in val_files:
        src = os.path.join(class_dir, file)
        dest = os.path.join(val_dir, class_name, file)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        shutil.copy(src, dest)

    for file in test_files:
        src = os.path.join(class_dir, file)
        dest = os.path.join(test_dir, class_name, file)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        shutil.copy(src, dest)

print("Dataset split into training, validation, and test sets.")


Dataset split into training, validation, and test sets.


DATA AUGMENTATION

In [7]:
import cv2
import os
import numpy as np

# Define the source directory containing your dataset
source_dir = 'C:\\Users\\afiqe\\Downloads\\DatasetSplit\\test'

# Define the target directory for augmented data
output_dir = 'C:\\Users\\afiqe\\Downloads\\DatasetNew\\test'

# Create the target directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# List the classes (subdirectories) in the source directory
class_names = os.listdir(source_dir)

# Define the augmentation factors
zoom_factor = 1.2  # Adjust the factor as needed
rotation_angles = [-10, 10]
blur_kernel_sizes = [3, 5]
sharpen_kernel = np.array([[-1, -1, -1],
                           [-1, 9, -1],
                           [-1, -1, -1]], dtype=np.float32)

# Function to apply and save data augmentation to an image
def apply_augmentation(image, class_name, output_dir, filename):
    if image is not None:
        # Save the original image
        original_save_path = os.path.join(output_dir, class_name, f'{filename}_original.jpg')
        cv2.imwrite(original_save_path, image)
        
        # Apply zooming
        h, w, _ = image.shape
        new_h, new_w = int(h * zoom_factor), int(w * zoom_factor)
        zoomed_image = cv2.resize(image, (new_w, new_h))
        h_diff, w_diff = (new_h - h) // 2, (new_w - w) // 2
        cropped_image = zoomed_image[h_diff:h_diff+h, w_diff:w_diff+w]
        save_path = os.path.join(output_dir, class_name, f'{filename}_{int(zoom_factor * 100)}_zoom.jpg')
        cv2.imwrite(save_path, cropped_image)
        
        for angle in rotation_angles:
            rotated_image = cv2.warpAffine(image, cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1), (w, h))
            save_path = os.path.join(output_dir, class_name, f'{filename}_{angle}_rotate.jpg')
            cv2.imwrite(save_path, rotated_image)
        
        for kernel_size in blur_kernel_sizes:
            blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
            save_path = os.path.join(output_dir, class_name, f'{filename}_{kernel_size}_blur.jpg')
            cv2.imwrite(save_path, blurred_image)
        
        sharpened_image = cv2.filter2D(image, -1, sharpen_kernel)
        save_path = os.path.join(output_dir, class_name, f'{filename}_sharpen.jpg')
        cv2.imwrite(save_path, sharpened_image)

        # Horizontal flip
        flipped_horizontal_image = cv2.flip(image, 1)
        save_path = os.path.join(output_dir, class_name, f'{filename}_flip_horizontal.jpg')
        cv2.imwrite(save_path, flipped_horizontal_image)

        # Vertical flip
        flipped_vertical_image = cv2.flip(image, 0)
        save_path = os.path.join(output_dir, class_name, f'{filename}_flip_vertical.jpg')
        cv2.imwrite(save_path, flipped_vertical_image)

# Loop through each class and apply data augmentation
for class_name in class_names:
    class_dir = os.path.join(source_dir, class_name)
    
    # Create subdirectories for each class in the output directory
    os.makedirs(os.path.join(output_dir, class_name), exist_ok=True)
    
    for filename in os.listdir(class_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):  # Add more extensions if needed
            image_path = os.path.join(class_dir, filename)
            
            image = cv2.imread(image_path)
            
            # Apply data augmentation and save augmented images
            apply_augmentation(image, class_name, output_dir, filename)

print("Data augmentation completed.")


Data augmentation completed.


DATA AUGMENTATION YOLOV8

In [ ]:
import cv2
import numpy as np
import os

def flip_horizontal(image, boxes):
    return cv2.flip(image, 1), boxes

def flip_vertical(image, boxes):
    return cv2.flip(image, 0), boxes

def rotate(image, boxes, angle=30):
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    image = cv2.warpAffine(image, M, (cols, rows))
    
    # Update bounding box coordinates
    new_boxes = []
    for box in boxes:
        x, y, w, h = box
        x, y = np.dot(M, [x, y, 1])[:-1]
        new_boxes.append([x, y, w, h])
    
    return image, new_boxes

def blur(image, boxes, ksize=(5, 5)):
    return cv2.GaussianBlur(image, ksize, 0), boxes

def crop(image, boxes, percent=0.2):
    rows, cols, _ = image.shape
    y1 = int(rows * percent)
    y2 = int(rows * (1 - percent))
    x1 = int(cols * percent)
    x2 = int(cols * (1 - percent))
    
    image = image[y1:y2, x1:x2]
    
    # Update bounding box coordinates
    new_boxes = []
    for box in boxes:
        x, y, w, h = box
        x, y = x - x1, y - y1
        new_boxes.append([x, y, w, h])
    
    return image, new_boxes

def read_annotations(annotation_path):
    with open(annotation_path, 'r') as f:
        lines = f.readlines()
    
    boxes = [list(map(float, line.strip().split()[1:])) for line in lines]
    return np.array(boxes)

def write_annotations(annotation_path, boxes):
    with open(annotation_path, 'w') as f:
        for box in boxes:
            f.write("0 " + " ".join(map(str, box)) + '\n')

def augment_data(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    
    for file_name in os.listdir(input_folder):
        if file_name.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_folder, file_name)
            annotation_path = os.path.join(input_folder, os.path.splitext(file_name)[0] + '.txt')
            
            image = cv2.imread(image_path)
            boxes = read_annotations(annotation_path)
            
            # Augmentation functions
            augmentations = [flip_horizontal, flip_vertical, rotate, blur, crop]
            
            for aug_func in augmentations:
                augmented_image, augmented_boxes = aug_func(image.copy(), boxes.copy())
                
                output_image_path = os.path.join(output_folder, f"{aug_func.__name__}_{file_name}")
                output_annotation_path = os.path.join(output_folder, f"{aug_func.__name__}_{os.path.splitext(file_name)[0]}.txt")
                
                cv2.imwrite(output_image_path, augmented_image)
                write_annotations(output_annotation_path, augmented_boxes)

if __name__ == "__main__":
    input_folder = "path/to/your/dataset"
    output_folder = "path/to/your/augmented/dataset"
    
    augment_data(input_folder, output_folder)


In [ ]:
import cv2
import numpy as np
import os

def draw_boxes(image, boxes, color=(0, 255, 0), thickness=2):
    for box in boxes:
        x, y, w, h = box
        x1, y1 = int(x - w / 2), int(y - h / 2)
        x2, y2 = int(x + w / 2), int(y + h / 2)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)

def visualize_augmented_images(input_folder, output_folder, num_images=5):
    for file_name in os.listdir(input_folder)[:num_images]:
        if file_name.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_folder, file_name)
            annotation_path = os.path.join(input_folder, os.path.splitext(file_name)[0] + '.txt')
            
            image = cv2.imread(image_path)
            original_boxes = read_annotations(annotation_path)
            
            # Draw bounding boxes on the original image
            original_image = image.copy()
            draw_boxes(original_image, original_boxes)
            
            cv2.imshow("Original Image", original_image)
            cv2.waitKey(0)
            
            # Show augmented images with bounding boxes
            for aug_func in augmentations:
                augmented_image_path = os.path.join(output_folder, f"{aug_func.__name__}_{file_name}")
                augmented_annotation_path = os.path.join(output_folder, f"{aug_func.__name__}_{os.path.splitext(file_name)[0]}.txt")
                
                augmented_image = cv2.imread(augmented_image_path)
                augmented_boxes = read_annotations(augmented_annotation_path)
                
                # Draw bounding boxes on the augmented image
                augmented_image_with_boxes = augmented_image.copy()
                draw_boxes(augmented_image_with_boxes, augmented_boxes)
                
                cv2.imshow(f"{aug_func.__name__} Augmentation", augmented_image_with_boxes)
                cv2.waitKey(0)

if __name__ == "__main__":
    input_folder = "path/to/your/dataset"
    output_folder = "path/to/your/augmented/dataset"
    
    visualize_augmented_images(input_folder, output_folder)
    cv2.destroyAllWindows()


FIND UNICODE

In [19]:
import os
import shutil
import codecs

def fix_non_utf8_characters(src_directory, dest_directory):
    # Create the destination directory if it doesn't exist
    os.makedirs(dest_directory, exist_ok=True)

    # List files in the source directory
    files = os.listdir(src_directory)

    for file in files:
        src_path = os.path.join(src_directory, file)
        
        if os.path.isfile(src_path):
            # Handle non-UTF-8 characters in the file name
            try:
                # Decode the filename using 'utf-8'
                decoded_filename = file.encode('latin-1').decode('utf-8')
                dest_path = os.path.join(dest_directory, decoded_filename)

                # Copy the file to the destination directory with the decoded filename
                shutil.copy(src_path, dest_path)
            except Exception as e:
                print(f"Error processing file: {file} - {e}")
                continue

if __name__ == "__main__":
    src_directory = "C:\\Users\\afiqe\\Downloads\\DatasetNew\\Dataset\\nasi lemak"
    dest_directory = "C:\\Users\\afiqe\\Downloads\\DatasetNew\\nasi lemak"

    # Call the function to copy files with non-UTF-8 characters
    fix_non_utf8_characters(src_directory, dest_directory)
